# dark-vessel-hunter
DTU Deep Learning project 29, group 80

## Required libraries installation
Run this in your terminal before executing this:

In [7]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Data

#### File imports for the data script

In [12]:
import config
import src.data.ais_downloader as ais_downloader
import src.data.ais_filtering as ais_filtering
import src.data.ais_reader as ais_reader
import src.data.ais_to_parquet as ais_to_parquet

#### Library imports for the data script

In [13]:
from tqdm import tqdm
from pathlib import Path
import pandas as pd
from datetime import date, timedelta

#### Set data preferences

In [14]:
START_DATE = "2025-11-01"
END_DATE   = "2025-11-03"

FOLDER_NAME = config.AIS_DATA_FOLDER
DELETE_DOWNLOADED_CSV = False
VERBOSE_MODE = True

VESSEL_AIS_CLASS = ("Class A", "Class B")

MIN_SEGMENT_LENGTH = 30     # datapoints
MAX_TIME_GAP_SEC = 30       # seconds

# Bounding Box to prefilter AIS data [lat_max, lon_min, lat_min, lon_max]
bbox = [57.58, 10.5, 57.12, 11.92]

# Polygon coordinates for precise Area of Interest (AOI) filtering (lon, lat)
polygon_coords = [
    (10.5162, 57.3500),  # coast top left (lon, lat)
    (10.9314, 57.5120),  # sea top left
    (11.5128, 57.5785),  # sea top right
    (11.9132, 57.5230),  # top right (Swedish coast)
    (11.9189, 57.4078),  # bottom right (Swedish coast)
    (11.2133, 57.1389),  # sea bottom right
    (11.0067, 57.1352),  # sea bottom left
    (10.5400, 57.1880),  # coast bottom left
    (10.5162, 57.3500),  # close polygon
]

#### Script

In [15]:
# --- Create paths ---
folder_path = Path(FOLDER_NAME)
folder_path.mkdir(parents=True, exist_ok=True)
csv_folder_path = folder_path / "csv"
csv_folder_path.mkdir(parents=True, exist_ok=True)
parquet_folder_path = folder_path / "parquet"
parquet_folder_path.mkdir(parents=True, exist_ok=True)

file_port_locations = folder_path / "port_locodes.csv"


# --- If you want to download all csv files before, uncomment the line below ---
# ais_downloader.download_multiple_ais_data(START_DATE, END_DATE, folder_path)

# --- Build the schedule of download string dates ---
dates = ais_downloader.get_work_dates(START_DATE, END_DATE, csv_folder_path, filter=False)

# --- Iterate with tqdm and download, unzip and delete ---
for day in tqdm(dates, desc=f"Processing data", unit="file" ):
    tag = f"{day:%Y-%m}" if day < date.fromisoformat("2024-03-01") else f"{day:%Y-%m-%d}"
    print(f"\nProcessing date: {tag}")

    # --- Download one day ---
    csv_path = ais_downloader.download_one_ais_data(day, csv_folder_path)
    
    # --- Load CSV into DataFrame ---
    df_raw = ais_reader.read_single_ais_df(csv_path, bbox, verbose=VERBOSE_MODE)
    # --- Optionally delete the downloaded CSV file ---
    if DELETE_DOWNLOADED_CSV: csv_path.unlink(missing_ok=True)
    
    # --- Filter and split ---
    # Filter AIS data, keeping Class A and Class B by default,
    df_filtered = ais_filtering.filter_ais_df(
        df_raw,
        polygon_coords=polygon_coords,
        allowed_mobile_types=VESSEL_AIS_CLASS,
        bbox=bbox,                          # select bbox 
        apply_polygon_filter=True,          # keep polygon filtering enabled boolean
        remove_zero_sog_vessels=False,      # use True/False to enable/disable 90% zero-SOG removal
        sog_in_knots=False,                 # convert SOG from knots in m/s (default) boolean
        port_locodes_path=file_port_locations,
        exclude_ports=True,                 # exclude port areas boolean 
        verbose=VERBOSE_MODE,               # verbose mode boolean
    )

    # --- Parquet conversion ---
    # Segment and save to Parquet by MMSI
    df_seg = ais_to_parquet.segment_ais_tracks(df_filtered, min_track_len=MIN_SEGMENT_LENGTH, max_time_gap_sec=MAX_TIME_GAP_SEC, verbose=VERBOSE_MODE)
    # Save segmented data to Parquet files
    ais_to_parquet.save_by_mmsi(df_seg, verbose=VERBOSE_MODE, output_folder=parquet_folder_path)

Processing data:   0%|          | 0/3 [00:00<?, ?file/s]


Processing date: 2025-11-01
Skipping 2025-11-01 download: already present in ais-data/csv folder
 Read AIS data: 988,647 rows within bbox,  241 unique vessels


Processing data:   0%|          | 0/3 [00:02<?, ?file/s]


KeyboardInterrupt: 

## Pre processing

#### File imports

#### Library imports for the pre processing script

#### Set pre processing preferences

In [ ]:
FOLDER_NAME = config.AIS_DATA_FOLDER
folder_path = Path(FOLDER_NAME)
parquet_folder_path = folder_path / "parquet"

SEGMENT_MAX_LENGTH = config.SEGMENT_MAX_LENGTH

NUMERIC_COLS = config.NUMERIC_COLS
# if u want to do it withouth a end date comment next line
TRAIN_START_DATE = config.TRAIN_START_DATE
TRAIN_END_DATE = config.TRAIN_END_DATE

TEST_START_DATE = config.TEST_START_DATE
TEST_END_DATE = config.TEST_END_DATE